In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, random_split

In [8]:
# Image preprocessing
transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the dataset
data_dir = '/content/drive/MyDrive/JPEGImages'
full_dataset = datasets.ImageFolder(data_dir, transform=transform)

# Split the dataset into training and validation sets
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)



In [9]:
# Load the pretrained ResNet50 model and modify the final layer
model = models.resnet50(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 50)  # Adjust this number based on the number of classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 187MB/s]


In [10]:
# Training model
def train_model(model, criterion, optimizer, num_epochs=10):
    model.train()  # Set the model to training mode
    start_time = time.time()  # Start time measurement

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        running_loss = 0.0
        running_corrects = 0

        for batch, (inputs, labels) in enumerate(train_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            # forward and backward
            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels)

            # Print loss every 20 steps
            if (batch + 1) % 20 == 0:
                print(f'Step {batch+1}, Loss: {loss.item():.4f}')

        epoch_loss = running_loss / train_size
        epoch_acc = running_corrects.double() / train_size
        print(f'Train Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}')

        # Validate the model
        model.eval()  # Set the model to evaluation mode
        val_loss = 0.0
        val_corrects = 0

        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            with torch.no_grad():
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

            val_loss += loss.item() * inputs.size(0)
            val_corrects += torch.sum(preds == labels)

        val_loss = val_loss / val_size
        val_acc = val_corrects.double() / val_size
        print(f'Validation Loss: {val_loss:.4f}, Acc: {val_acc:.4f}')

    time_elapsed = time.time() - start_time
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')



In [11]:
train_model(model, criterion, optimizer, num_epochs=10)

Epoch 1/10
Step 20, Loss: 3.7680
Step 40, Loss: 3.4049
Step 60, Loss: 3.1339
Step 80, Loss: 2.8711
Step 100, Loss: 2.6875
Step 120, Loss: 2.3534
Step 140, Loss: 1.9989
Step 160, Loss: 1.7583
Step 180, Loss: 1.7182
Step 200, Loss: 1.6432
Step 220, Loss: 1.5232
Step 240, Loss: 1.4667
Train Loss: 2.4080, Acc: 0.5225
Validation Loss: 1.3839, Acc: 0.7324
Epoch 2/10
Step 20, Loss: 1.3430
Step 40, Loss: 0.8445
Step 60, Loss: 1.1191
Step 80, Loss: 1.2687
Step 100, Loss: 1.0659
Step 120, Loss: 1.3315
Step 140, Loss: 1.2253
Step 160, Loss: 0.9844
Step 180, Loss: 0.9202
Step 200, Loss: 0.8026
Step 220, Loss: 1.3355
Step 240, Loss: 0.7874
Train Loss: 1.0696, Acc: 0.7990
Validation Loss: 0.9213, Acc: 0.8193
Epoch 3/10
Step 20, Loss: 0.7431
Step 40, Loss: 0.8818
Step 60, Loss: 0.7805
Step 80, Loss: 0.7076
Step 100, Loss: 0.8156
Step 120, Loss: 0.6710
Step 140, Loss: 0.5002
Step 160, Loss: 0.7393
Step 180, Loss: 0.8640
Step 200, Loss: 0.5640
Step 220, Loss: 0.6734
Step 240, Loss: 0.8259
Train Loss: 0